In [15]:
import pandas as pd

In [16]:
data = pd.read_csv(r'C:\Users\dhruv\OneDrive\Documents\Python Projects\NLP\datasets\train.csv')

In [17]:
data.head()

,sms,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...\n,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [18]:
data[100:101]['sms']

100    Please don't text me anymore. I have nothing e...
Name: sms, dtype: object

In [19]:
from nltk.stem import WordNetLemmatizer

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd  # Assuming pandas is used for DataFrame handling
from tqdm import tqdm  # Optional: Progress bar for large datasets

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [24]:
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    
    words = word_tokenize(text.lower(), language = "english")
    return " ".join([lemmatizer.lemmatize(word) for word in words if word not in stop_words and word.isalnum()])

tqdm.pandas()

In [25]:
data['processed_sms'] = data['sms'].progress_apply(preprocess_text)

100%|██████████| 5574/5574 [02:15<00:00, 41.02it/s]


In [26]:
tfidf_vectorizer = TfidfVectorizer(max_features=2500, stop_words='english', ngram_range=(1, 2), min_df=5)

tfidf_matrix = tfidf_vectorizer.fit_transform(data['processed_sms']).toarray()

tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

print("TF-IDF Matrix Shape:", tfidf_matrix.shape)
print("Sample TF-IDF Feature Names:", tfidf_feature_names[:10])

TF-IDF Matrix Shape: (5574, 1970)
Sample TF-IDF Feature Names: ['0800' '08000839402' '08000930705' '0870' '0870 national' '08707509020'
 '08707509020 20p' '08712300220' '08712300220 quoting' '08712460324']


In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(tfidf_matrix, data['label'], train_size = 0.7, random_state = 42)

In [29]:
from sklearn.naive_bayes import MultinomialNB

In [30]:
spam_model = MultinomialNB().fit(X_train, Y_train)

In [31]:
Y_pred = spam_model.predict(X_test)

In [32]:
from sklearn.metrics import accuracy_score

In [33]:
accuracy = accuracy_score(Y_test,Y_pred)
print(accuracy*100)

97.60908547519426


In [34]:
from sklearn.model_selection import KFold
k = 5 
kf = KFold(n_splits=k, shuffle=True, random_state=42)
from sklearn.model_selection import cross_val_score 
scores = cross_val_score(spam_model, X_train, Y_train, cv=kf, scoring='r2')

In [35]:
import numpy as np 

average_r2 = np.mean(scores) 

print(f"R² Score for each fold: {[round(score, 4) for score in scores]}")
print(f"Average R² across {k} folds: {average_r2:.2f}")

R² Score for each fold: [0.7915, 0.7206, 0.7199, 0.7962, 0.7958]
Average R² across 5 folds: 0.76
